In [36]:
import pytesseract
import platform
if platform.system() == "Windows":
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\B92383\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 2 --psm 12 -l fra+eng'

In [37]:
import cv2
import numpy as np

# Etape 1
### Standariser l'image

In [38]:
def resize_image(image):
    """
    Resize the input image to a fixed size of (1946, 1080).

    Parameters:
    - image: Input image (OpenCV format)

    Returns:
    - Resized image
    """
    # Resize the image to the fixed size
    resized_image = cv2.resize(image, (460, 800))

    return resized_image




def biggestRectangle(contours):
    max_area = 0
    indexReturn = -1
    for index in range(len(contours)):
        i = contours[index]
        area = cv2.contourArea(i)
        if area > 100:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.1 * peri, True)
            if area > max_area:
                max_area = area
                indexReturn = index
    return indexReturn


def image_brightness(img):
    """
    Process the input image by converting color space, applying gamma correction,
    and applying binary thresholding.

    Parameters:
    - img: Input image (OpenCV format)

    Returns:
    - Processed image
    """
    # Convert color space from BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    # Apply gamma correction
    invGamma = 1.0 / 0.3
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    gray = cv2.LUT(gray, table)

    # Apply binary thresholding
    _, thresh1 = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

    return thresh1


def process_image(img):
    
    thresh1 = image_brightness(img)
    
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]

    indexReturn = biggestRectangle(contours)
    hull = cv2.convexHull(contours[indexReturn])

    mask = np.zeros_like(img)
    cv2.drawContours(mask, contours, indexReturn, 255, -1)
    out = np.zeros_like(img)
    out[mask == 255] = img[mask == 255]
    

    (y, x, _) = np.where(mask == 255)
    (topy, topx) = (np.min(y), np.min(x))
    (bottomy, bottomx) = (np.max(y), np.max(x))
    out = img[topy:bottomy + 1, topx:bottomx + 1, :]
    out = resize_image(out)
    

    return out




# Define a function to trim and save an image
def trim_and_save(region_key, region_points, image):
    x, y, w, h = cv2.boundingRect(np.array(region_points))
    cropped_image = image[y:y+h, x:x+w]
    cv2.imwrite(f"./img/{region_key}_cropped.jpg", cropped_image)



def image2text(cropped_image_path):
    text = ''
    
    cropped_image = cv2.imread(cropped_image_path)

    text = pytesseract.image_to_string(cropped_image, config=custom_config)

    if len(text)!=0:
        return text
    else:
        print("no text detected")


        




In [39]:
# Load your image
image_path = "/home/apprenant/Bureau/ocr/carte-grise-specimen.jpg"
# image = Image.open(image)
frame = cv2.imread(image_path)
print(frame.shape)
# print(frame)
frame = process_image(frame)

print(frame.shape)

(2656, 1472, 3)
(800, 460, 3)


# Etape 2 :
### Trouver les points de chaque element dans la photo

In [40]:
import numpy as np
import cv2

def mouse_drawing(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Left click")
        circles.append((x, y))

circles = []

# Load your image
# image_path = "img\carte-grise-specimen.jpg"
# frame = cv2.imread(image_path)

cv2.namedWindow("Frame")
if len(circles) < 4:
    cv2.setMouseCallback("Frame", mouse_drawing)

while True:
    for center_position in circles:
        cv2.circle(frame, center_position, 2, (0, 0, 255), -1)
    
    points = np.array(circles)
    if len(circles) >= 4:
        cv2.polylines(frame, np.int32([points]), 1, (255, 255, 255))

    cv2.imshow("Frame", frame)
    

    key = cv2.waitKey(1)
    if key == 27:
        break
    elif key == ord("d"):
        circles = []

cv2.destroyAllWindows()
print(circles)


[]


In [41]:
regions_dict = {
    'points_N_immatriculation':[(31, 35), (31, 46), (100, 49), (101, 35)],
    'points_titre_document': [(113, 0), (114, 15), (339, 15), (336, 0)],
    'points_d_2': [(38, 242), (38, 262), (359, 262), (359, 242)],
    'points_E': [(288, 276), (289, 293), (412, 295), (412, 275)],
    'points_Date_de_1er_immarticulation': [(145, 37), (146, 50), (228, 51), (226, 36)]
}

# Accessing values for a specific key
print(regions_dict['points_titre_document'][0])

(113, 0)


# Etape 3
### couper les elements et les save 

In [42]:
import cv2

# Iterate through regions and trim/save images
for region_key, region_points in regions_dict.items():
    trim_and_save(region_key, region_points, frame)

# Etape 4
### Passer le PCR pour chaque image

In [43]:
while True:
    for region_key, value in regions_dict.items():
        cv2.rectangle(frame, value[0], value[2], (0,0,255),2,2)
         # Load the saved cropped image
        cropped_image_path = f"{region_key}_cropped.jpg"
        # cropped_image = cv2.imread(cropped_image_path)
        word_found = image2text(cropped_image_path)
        # print(f"{region_key} : {word_found}")
        cv2.putText(frame, str(word_found), value[0],1,1,(0,0,255),1)

    
 
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
  
cv2.destroyAllWindows()

In [44]:
liste_var = {}
for region_key, value in regions_dict.items():
        cv2.rectangle(frame, value[0], value[2], (0,0,255),2,2)
         # Load the saved cropped image
        cropped_image_path = f"./img/{region_key}_cropped.jpg"
        # cropped_image = cv2.imread(cropped_image_path)
        word_found = image2text(cropped_image_path)
        liste_var[region_key]=word_found.strip()

In [45]:
liste_var

{'points_N_immatriculation': 'AB-123-CD',
 'points_titre_document': "CERTIFICAT D'IMNATAICULATION",
 'points_d_2': 'VERSION',
 'points_E': 'WFSIV2009ASIV2009',
 'points_Date_de_1er_immarticulation': '05/01/1996'}

In [46]:
from PyPDF2 import PdfWriter, PdfReader
from io import BytesIO
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

buffer = BytesIO()

# create a new PDF with Reportlab
p = canvas.Canvas(buffer, pagesize=A4)
dict_pos = {
    'points_N_immatriculation':[50, 705],
    'points_d_2': [50, 630],
    'points_E': [50, 610],
}
for region_key in dict_pos.keys():
    p.drawString(*dict_pos[region_key], liste_var[region_key])
p.showPage()
p.save()

#move to the beginning of the StringIO buffer
buffer.seek(0)
newPdf = PdfReader(buffer)

# #######DEBUG NEW PDF created#############
# pdf1 = buffer.getvalue()
# open('pdf1.pdf', 'wb').write(pdf1)
#########################################
# read your existing PDF
existingPdf = PdfReader(open('cerfa_13750-07.pdf', 'rb'))
output = PdfWriter()
# add the "watermark" (which is the new pdf) on the existing page
page = existingPdf.pages[0]
page.merge_page(newPdf.pages[0])
output.add_page(page)
# finally, write "output" to a real file
outputStream = open('output.pdf', 'wb')
output.write(outputStream)
outputStream.close()